# 필요 패키지 설치

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 유사도 계산

In [ ]:
# 데이터 경로 (사용자가 수정 가능)
DATA_PATH = "./data/sentence_pairs.csv"

# 유사도 계산에 사용할 모델
SIM_MODEL_NAME = "jhgan/ko-sroberta-multitask"

def compute_similarity(df):
    """
    adult_sentence와 kids_sentence 간 코사인 유사도를 계산하여 similarity_score 컬럼에 추가

    Args:
        df (pd.DataFrame): CSV에서 불러온 데이터프레임

    Returns:
        pd.DataFrame: similarity_score 추가된 데이터프레임
    """
    print("✓ 유사도 모델 로딩 중...")
    model = SentenceTransformer(SIM_MODEL_NAME)

    similarities = []
    for _, row in df.iterrows():
        sent1 = str(row["adult_sentence"])
        sent2 = str(row["kids_sentence"])

        emb1 = model.encode(sent1, convert_to_tensor=True)
        emb2 = model.encode(sent2, convert_to_tensor=True)

        sim = util.pytorch_cos_sim(emb1, emb2).item()
        similarities.append(sim)

    df["similarity_score"] = similarities
    return df

if __name__ == "__main__":
    print("✓ 데이터 불러오는 중...")
    df = pd.read_csv(DATA_PATH)

    if "adult_sentence" not in df.columns or "kids_sentence" not in df.columns:
        raise ValueError("CSV에는 'adult_sentence', 'kids_sentence' 컬럼이 반드시 있어야 합니다.")

    df = compute_similarity(df)

    # 저장
    df.to_csv(DATA_PATH, index=False)
    print("csv 저장")